<a href="https://colab.research.google.com/github/Serge3leo/temp-cola/blob/main/ruSO/1615333-Найти-длину-строки-от-big-факториала-в-питоне.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Найти длину строки от Big факториала в питоне

К моему [ответу](https://ru.stackoverflow.com/a/1615337/430734) на [вопрос](https://ru.stackoverflow.com/q/1615333/430734).

In [1]:
import math

def count(n):
    return 1 + int(math.lgamma(n + 1)/math.log(10))

assert count(5) == 3
assert count(50) == 65
assert count(500) == 1135
assert count(5000) == 16326
assert count(50000) == 213237
assert count(500000) == 2632342
assert count(5000000) == 31323382
assert count(50000000) == 363233781
for n in range(10):
    assert count(n) == len(str(math.factorial(n)))

## Вопрос оценки точности

Т.е. найти $M | \forall n < M \ count(n) = gcount(n)$, где $gcount(n)$
вычисляется с высокой точностью.

Для функции `lgamma()` реализованной `glibc` в [19.7 Known Maximum Errors in Math Functions](https://www.gnu.org/software/libc/manual/html_node/Errors-in-Math-Functions.html) обещают 4 ULP для i686/x86_64 и 3 ULP для AArch64(ARM64).
На BSD/macOS точность немного лучше.

Наверное, достаточно сравнить с результат с
`1 + gmpy2.floor(gmpy2.lgamma(n + 1)[0]/gmpy2.log(10))` для таких
$n$, у которых $\log_{10} n!$ близок к целому числу.

Поиск по числам вида $b^{i}!$ нашёл $M_0 = 6177^3 =
2.35685467233 \times 10^{11}$, для
которого $M_0! \approx 0.9998010 \times 10^{2577936672564}$ или
$$\frac{\left\lceil \log_{10} M_0! \right\rceil - \log_{10} M_0!}{\log_{10} M_0!}
  \approx 2^{-53.5} \approx 10^{-16.1}$$
, что меньше точности представления результата функции `math.lgamma()`.

В принципе, прямая проверка $2 \times 10^{11}$ чисел

In [2]:
import time

import gmpy2
import numpy as np

gmpy2.set_context(gmpy2.context(precision=2048))
# gmpy2.set_context(gmpy2.context(precision=(2048+1024)))

def gcount(n):
    return 1 + gmpy2.floor(gmpy2.lgamma(n + 1)[0]/gmpy2.log(10))
    
def ncount(n, chunk=1_000_000):
    if n <= 1:
        return 1
    slog = 0.
    st = 2
    while st <= n:
        slog += np.sum(np.log10(np.arange(st, min(st + chunk, n + 1))))
        st += chunk
    return 1 + int(slog)    
    # return (np.ceil(np.sum(np.log10(np.arange(2, n + 1))))
    #         if n > 1 else 1)

def count(n):
    return 1 + int(math.lgamma(n + 1)/math.log(10))

for cnt in [ncount, count, gcount]:
    dt = -time.time()
    assert cnt(5) == 3
    assert cnt(50) == 65
    assert cnt(500) == 1135
    assert cnt(5000) == 16326
    assert cnt(50000) == 213237
    assert cnt(500000) == 2632342
    assert cnt(5000000) == 31323382
    assert cnt(50000000) == 363233781
    for n in range(10):
        assert cnt(n) == len(str(math.factorial(n)))
    dt += time.time()
    print(dt, cnt)

0.7460460662841797 <function ncount at 0x10c71fce0>
2.288818359375e-05 <function count at 0x10c71f100>
0.019666194915771484 <function gcount at 0x10c71f2e0>


In [3]:
for m0 in [63342189195264, 45767944570401, 41407371740736,
           6478348728125, 4335760898001, 235685467233]:
    print(f"{m0=}")
    for i in range(-10, 11):
        mlg = math.lgamma(m0 + i)
        glg = gmpy2.lgamma(m0 + i)[0]
        dlg = mlg - glg
        print(f"{i:3}, {float(dlg/math.ulp(mlg)):5.2f} ULP, {float(dlg)}")

m0=63342189195264
-10,  0.58 ULP, 0.14520661999120388
 -9,  0.46 ULP, 0.11563390094675098
 -8,  0.34 ULP, 0.08606118190228232
 -7,  0.23 ULP, 0.05648846285779786
 -6,  0.11 ULP, 0.026915743813297612
 -5,  0.99 ULP, 0.2473430247687816
 -4, -0.13 ULP, -0.032229694275750245
 -3,  0.75 ULP, 0.18819758667970216
 -2,  0.63 ULP, 0.15862486763513875
 -1,  0.52 ULP, 0.12905214859055958
  0,  0.40 ULP, 0.0994794295459646
  1,  1.28 ULP, 0.31990671050135383
  2,  0.16 ULP, 0.04033399145672731
  3,  0.04 ULP, 0.010761272412084974
  4,  0.92 ULP, 0.23118855336742686
  5, -0.19 ULP, -0.04838416567724705
  6,  0.69 ULP, 0.17204311527806326
  7,  0.57 ULP, 0.14247039623335778
  8,  0.45 ULP, 0.11289767718863651
  9,  0.33 ULP, 0.08332495814389945
 10,  1.22 ULP, 0.3037522390991466
m0=45767944570401
-10,  0.39 ULP, 0.09772674990341998
 -9,  0.57 ULP, 0.1431216881090323
 -8,  0.75 ULP, 0.18851662631462276
 -7, -0.06 ULP, -0.016088435479808604
 -6,  0.12 ULP, 0.02930650272573817
 -5,  0.30 ULP, 0.0747014